# MACD (Moving Average Convergence/Divergence) crossover

Use MACD to determine when to buy or sell stock

In [ ]:
from datetime import date, datetime, timedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pandas_datareader.data as datareader

plt.style.use('fivethirtyeight')


In [ ]:
# Load data
SYMBOL = 'AAPL'

start = datetime.now() - timedelta(days=365)
df = datareader.get_data_yahoo(SYMBOL, start=start)
## if need, set date to be the index
# df = df.set_index(pd.DatetimeIndex(df['Date'].values))
  
df

In [ ]:
plt.figure(figsize=(12.2, 4.5))
plt.plot(df['Close'], label="close")
plt.title("Close preice history")
plt.xticks(rotation=45)
plt.xlabel('Date')
plt.ylabel('Price ($)')

In [ ]:
# Calculate the MACF and signal line indicators
# Calculate the short term exponential moving average (EMA)

ShortEMA = df.Close.ewm(span=12, adjust=False).mean()

# Calculate the long term exponential moving average (EMA)
LongEMA = df.Close.ewm(span=26, adjust=False).mean()

# Calculate the MACD
MACD = ShortEMA - LongEMA

# Calculate the singal line
signal = MACD.ewm(span=9, adjust=False).mean()



In [ ]:
plt.figure(figsize=(12.2, 4.5))
plt.plot(df.index, MACD, label = 'AAPL MACD', color='red')
plt.plot(df.index, signal, label='Signal Line', color='blue')
plt.xticks(rotation=45)
plt.legend(loc='upper left')

In [ ]:
df['MACD'] = MACD
df['Signal Line'] = signal


In [ ]:
def buy_sell(signal):
    Buy =[]
    Sell=[]
    flag = -1
    for i in range(0, len(signal)):
        if signal['MACD'][i] > signal['Signal Line'][i]:
            Sell.append(np.nan)
            if flag != 1:
                Buy.append(signal['Close'][i])
                flag = 1
            else:
                Buy.append(np.nan)
        elif signal['MACD'][i] < signal['Signal Line'][i]:
            Buy.append(np.nan)
            if flag != 0:
                Sell.append(signal['Close'][i])
                flag = 0
            else:
                Sell.append(np.nan)
        else:
            Buy.append(np.nan)
            Sell.append(np.nan)
    return (Buy, Sell)

In [ ]:
a = buy_sell(df) # (Buy, Sell)
df['Buy_Signal_Price'] = a[0]
df['Sell_Signal_Price'] = a[1]


df

In [ ]:
plt.figure(figsize=(12.2, 4.5))
plt.scatter(df.index, df['Buy_Signal_Price'], color='red', label='Buy', marker='^', alpha =1)
plt.scatter(df.index, df['Sell_Signal_Price'], color='green', label='Sell', marker='v', alpha =1)

plt.plot(df['Close'], label='Close Price', alpha=0.35)
plt.title('Close Price Buy & Sell Signals')
plt.xticks(rotation=45)
plt.legend(loc='upper left')
plt.xlabel('Date')
plt.ylabel('Close Price ($)')